#Simulation to determine sample size

In [4]:
from tqdm import tqdm
import random
import statistics


"""
Execute the monte carlo simulation.
"""
if __name__ == "__main__":

    maxIterations = 1000
    cellSize = 50   
    
    # Distribution of opinion of GM food is 24% (pro) versus 76% (con)
    distribution = [1]*24 + [0]*76
    # Distribution of previous sessions for other experiments
    priorSession = [1]*2 + [2]*8 + [3]*7 + [4]*6 + [5]*7 + [6]*11 + [7]*24
        
    totalSample = []
    totalSession = []    
    totalRedundant = []
        
    for iteration in tqdm(range(maxIterations)):
        
        conditions = {
            'pro' : 0,
            'con' : 0,
            'maj' : 0,
            'min' : 0
            'mix' : 0
        }
                       
        n = m = 0
    
        while all(value >= cellSize for value in conditions.values()) is False:
            
            # Generate a session with max 7 participants
            sessionSize = priorSession[random.randint(0, len(priorSession)-1)]    
            priorOpinion = []
            redundant = 0
            
            i = 0
            while i < sessionSize:        
                priorOpinion.append(distribution[random.randint(0, 99)])
                i += 1
            
            con = [0]*priorOpinion.count(0)
            pro = [1]*priorOpinion.count(1)
            
            j = 0
            while j < sessionSize:      
                
                # Two participants with positive opinion
                if len(pro) > 1:
                    
                    mixed = conditions['mix'] if 'mix' in conditions else min(conditions['maj'], conditions['min']) 

                    # Check if positive opinion condition is smaller than the mixed opinion condition
                    if conditions['pro'] < mixed:
                        del pro[0:2]
                        conditions['pro'] += 2
                    
                    else:
                        if len(con) == 0:
                            del pro[0:2]
                            conditions['pro'] += 2
                        else:
                            del con[0:1]
                            del pro[0:1]
                            if 'mix' in conditions:
                                conditions['mix'] += 2
                            else:
                                if conditions['maj'] < conditions['min']:
                                    conditions['maj'] += 2
                                else:
                                    conditions['min'] += 2
                            
                # One participant with positive opinion            
                elif len(pro) == 1:
                    
                    if len(con) > 0:
                        del con[0:1]
                        del pro[0:1]
                        if 'mix' in conditions:
                                conditions['mix'] += 2
                        else:
                            if conditions['maj'] < conditions['min']:
                                conditions['maj'] += 2
                            else:
                                conditions['min'] += 2
                    else:
                        redundant += 1
                
                # Only participants with negative opinion
                if len(con) > 1:
                    del con[0:2]
                    conditions['con'] += 2
                    
                j += 2

            redundant += sum([x - cellSize for x in [v for (k,v) in conditions.items() if v > cellSize]])
            redundant += len(con) + len(pro)

            n += sessionSize
            m += 1
            
        totalSample.append(n)
        totalSession.append(m)
        totalRedundant.append(redundant)
    

    print('\nNumber of iterations: ', maxIterations)
    print('Number of cells: ', len(conditions))  
    print('Number of participants per cell: ', cellSize)    
    print('Sample size: (M = {:.2f}, SD = {:.2f})'.format(statistics.mean(totalSample), statistics.stdev(totalSample)))
    print('Number of sessions: (M = {:.2f}, SD = {:.2f})'.format(statistics.mean(totalSession), statistics.stdev(totalSession)))
    print('Number of redundant participants: (M = {:.2f}, SD = {:.2f})'.format(statistics.mean(totalRedundant), statistics.stdev(totalRedundant)))


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1024.93it/s]



Number of iterations:  1000
Number of cells:  4
Number of participants per cell:  50
Sample size: (M = 445.77, SD = 38.56)
Number of sessions: (M = 87.40, SD = 8.49)
Number of redundant participants: (M = 192.77, SD = 33.82)
